Jupyter notebook that maps optometrist and ophthalmologist locations across the United States, 
then draws “zones” of eyecare provider access based on driving distances calculated using Mapbox 
isochrone API for different geographies of interest (ie. state, county, zip code, census tract).
these zones are visualized on a choropleth map by a 'coverage score', a value which represents 
the percentage of the chosen geography of interest that can access a provider.

## limitaions of the program:
the mapbox API has a limit of 300 requests per minute and will break if that limit is exceeded.
to get around this limition there is a timer function set so to ensure the limit is not exceeded, 
however there are over 90,000 providers in our dataset and this method is time consuming. 
It is therefore recommended to subset the dataset for the specific geography and classification of interest.

In [4]:
#importing necessary libraries
import requests
#from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
#import matplotlib
#from matplotlib import pyplot as plt
#import seaborn as sns
#plt.style.use('default')
import geopandas as gpd
#import geodatasets
import ast
from shapely import wkt
#from pyproj import Geod
import contextily
from collections import defaultdict
from shapely.ops import unary_union
pd.set_option('display.max_columns', None)
#import pyarrow as pyarrow
import fastparquet as fastparquet
import plotly.express as px
from keplergl import KeplerGl

In [5]:
# importing vision providers dataset accessible on github
providers = pd.read_parquet("../notebooks1/vision_providers_minimal.parquet")

# using taxonomy codes to categorize providers (accessible to view here: 
# https://data.cms.gov/provider-characteristics/medicare-provider-supplier-enrollment/medicare-provider-and-supplier-taxonomy-crosswalk/data
# )
def classify_provider(taxonomy):
    codes = taxonomy.split('|')
    for code in codes:
        if code.startswith('152'):
            return 'Optometry'
        if code.startswith('207'):
            return 'Ophthalmology'
        if code.startswith('156'):
            return 'Others'
    return 'Unknown'
providers['Classification'] = providers['Taxonomy'].apply(classify_provider)

In [14]:
# cleaning dataset
providers.dropna(subset=['Latitude'], inplace=True)

providers2 = providers[providers["Entity Type Code"] == 1]

providers2.reset_index(inplace=True)
providers2.drop(columns=['index', 'Entity Type Code', 'Replacement NPI', 'Employer Identification Number (EIN)',
                         'Provider Organization Name (Legal Business Name)', 'Provider Middle Name', 
                         'Provider Business Mailing Address Country Code (If outside U.S.)', 'Provider Gender Code',
                         'Provider Name Suffix Text', 'Provider Other Organization Name', 'Provider Other Organization Name Type Code',
                         'Provider Other Last Name', 'Provider Other First Name', 'Provider Other Middle Name', 'Provider Other Name Prefix Text', 
                         'Provider Other Name Suffix Text', 'Provider Other Credential Text', 'Provider Other Last Name Type Code', 
                         'Provider Business Mailing Address Postal Code', 'Authorized Official Last Name', 'Authorized Official First Name', 
                         'Authorized Official Middle Name', 'Authorized Official Title or Position', 'Authorized Official Telephone Number', 
                         'Certification Date', 'Taxonomy'], inplace=True)

# set up empty columns for the api function later
providers2['polygons'] = None
providers2['list of dict'] = None

# seperating optometrists and ophthalmologists
optom = providers2[providers2['Classification'] == 'Optometry']
print(len(optom))
opht = providers2[providers2['Classification'] == 'Ophthalmology']
print(len(opht))

# getting number of providers per location
unique_optom = pd.DataFrame(optom['Full Address'].value_counts(dropna=False))
optom_merged = optom.merge(unique_optom, how='left', on=['Full Address'])
optom['number of providers at this location'] = optom_merged['count']

unique_opht = pd.DataFrame(opht['Full Address'].value_counts(dropna=False))
opht_merged = opht.merge(unique_opht, how='left', on=['Full Address'])
opht['number of providers at this location'] = opht_merged['count']

# dropping rows with duplicate addresses so that only one provider per address is kept. 
# reducing the number of entries in the dataframe and therefore the number of requests sent to the api
optom1 = optom.drop_duplicates(subset=['Full Address'], ignore_index=True)
opht1 = opht.drop_duplicates(subset=['Full Address'], ignore_index=True)

# creating geodataframes
optom_gdf = gpd.GeoDataFrame(
   optom1, geometry=gpd.points_from_xy(optom1.Longitude, optom1.Latitude), crs="EPSG:4326")

opht_gdf = gpd.GeoDataFrame(
   opht1, geometry=gpd.points_from_xy(opht1.Longitude, opht1.Latitude), crs="EPSG:4326")

# you can chose to subset the dataframes for a specific state
optom_gdf = optom_gdf[optom_gdf["Provider Business Mailing Address State Name"] == 'IL']
opht_gdf = opht_gdf[opht_gdf["Provider Business Mailing Address State Name"] == 'IL']

optom_gdf.reset_index(inplace=True)
opht_gdf.reset_index(inplace=True)

/tmp/ipykernel_2667306/3316137378.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  providers2.drop(columns=['index', 'Entity Type Code', 'Replacement NPI', 'Employer Identification Number (EIN)',
/tmp/ipykernel_2667306/3316137378.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  providers2['polygons'] = None
/tmp/ipykernel_2667306/3316137378.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

60016
24735


In [15]:
# setting up geography datasets
# datasets for state and county level can be found on github. census tract datasets
# for oklahoma and illinois are also on github. if you want to see census tract data
# for a different state, you can download the dataset from here: 
# https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html#list-tab-790442341

# # state level
# state = gpd.read_file('../notebooks1/US_State_Boundaries.zip')
# state = state.to_crs("EPSG:4326")
# state.drop(columns=['FID', 'OBJECTID', 'ORDER_ADM', 'MONTH_ADM', 'DAY_ADM',
#                     'PRIM_MILES', 'Shape_Leng', 'Shape__Are', 'Shape__Len'], inplace=True)
# state_optom = state
# state_opht = state

# # county level
# county = gpd.read_file('../notebooks1/USA_Counties_(Generalized).zip')
# county = county.to_crs("EPSG:4326")
# county = county[['NAME', 'STATE_NAME', 'STATE_FIPS', ', CNTY_FIPS', 'FIPS', 'SQMI',
#                 'geometry']]
# county_optom = county
# county_opht = county

# census tract level example
ILtracts = gpd.read_file('../notebooks1/tl_2020_17_tract (3).zip')
ILtracts = ILtracts.to_crs("EPSG:4326")
ILtracts.drop(columns=['NAME', 'TRACTCE', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 
                       'INTPTLAT', 'INTPTLON'], inplace=True)
ILtracts_opt = ILtracts
ILtracts_opht = ILtracts

In [16]:
# function that feeds values from the longitude and latitude columns in providers dataframes
# to the api to get driving distance isochrones. works per row

def polygonf(row):
    """
    takes longitude and latitude data from their respective columns in vision providers dataframe
    requests mapbox api using that data outputs polygon data as a column in vpdf
    """
    apierror = "Failed to make the API request"
    features = 'no features'
    long = row['Longitude']
    lat = row['Latitude']
# defining mapbox api parameters. adjusting the number in the minutes string will change the distance. 
# check 'https://docs.mapbox.com/api/navigation/isochrone/' for full documentation
    apiurl = 'https://api.mapbox.com/isochrone/v1/mapbox/driving-traffic/'
    coord = '%2C'
    minutes ='?contours_minutes=15&polygons=true&denoise=1&'
    token = 'access_token=sk.eyJ1IjoibXlsZXNuZGlyaXR1IiwiYSI6ImNsanllY2hueTAwbXcza3JraTBkc2Z2bzIifQ.rxgKQl5MA6GIkKLbc4nAvA'
    Mapbox = apiurl + str(long) + coord + str(lat) + minutes + token

    # requesting data from api
    data_response = requests.get(Mapbox)

# Check if the request was successful (status code 200)
    if data_response.status_code == 200:
        full_json = data_response.json()
       
        if "features" in full_json:
            polygonz = full_json['features'][0]['geometry']

            return polygonz
        else:
            return features
    else:
        return apierror

In [ ]:
# timer function using optom_gdf as an example
# if you have a lot of data, you can make a copy of this notebook and run another dataset simultaneously.
# for this you will need to make a new API key here: https://docs.mapbox.com/playground/isochrone/

import time

chunk_size = 300
delay_seconds = 63

total_rows = len(optom_gdf)
num_chunks = (total_rows // chunk_size) + 1

current_chunk = 0

while current_chunk < num_chunks:
    start_index = current_chunk * chunk_size
    end_index = min((current_chunk + 1) * chunk_size, total_rows)

    # Process the chunk of the DataFrame
    chunk = optom_gdf.iloc[start_index:end_index]
    print(f'Processing chunk: {current_chunk+1}/{num_chunks}')
    print(f'Chunk index: {chunk.index}')
    chunk['polygons'] = chunk.apply(polygonf, axis=1)
    optom_gdf.iloc[start_index:end_index] = chunk

    # Print progress
    print(f"Processed chunk: {current_chunk+1}/{num_chunks}")

    # Wait for the specified time delay
    time.sleep(delay_seconds)

    # Move to the next chunk
    current_chunk += 1


Processing chunk: 1/7
Chunk index: RangeIndex(start=0, stop=300, step=1)


In [ ]:
# now you should have a dataframe with providers as well as coordinates that represent the driving distance isochrones

# here, i recommend saving the new dataframe as a csv as a precaution 
# this will save you from having to run the timer function again in case jupyter crashes

optom_gdf.to_csv('optom_gdf')

#converting the api data to active geometry
optom_gdf['isochrones'] = [shape(value) if isinstance(value, dict) else None for value in optom_gdf['polygons']]
optom_gdf.set_geometry('isochrones', inplace=True)
optom_gdf.set_crs("EPSG:4326", inplace=True)
print(optom_gdf.crs)

# # test
# ax = ILtracts.plot(figsize=(10, 10), color='white', linestyle=':', edgecolor='black')
# optom_gdf.plot(ax=ax, alpha = 0.3)

In [ ]:
# the next 3 cells make up the function that compares the isochrone data to the chosen geography data to calculate accessibility scores
# you should be able to run this without changing anything

#cdf == gdf of geometry area which you want to get the coverage score
#idf == gdf of points for which you want to get an isochrone and dictionary of percent of overlap in cdf areas
#scoreString = a string that is the name of the coverage score column you choose
#dictString = a string that is the name of the dictionary column you choose
#cgs == a string that is the name of the geometry column in the cdf
#igs == a string that is the name of the geometry column in the idf
#key = a string that is the name of the column in the cdf that you want to represent the key of the dictionary

def appendADS(cdf, idf, scoreString, dictString, cgs, igs, key):
    #api = MapboxAPI()
    cdf[scoreString] = 0.0
    cdf['polygons in census'] = np.empty((len(cdf), 0)).tolist()
    cdf['area coverage of each polygon in census'] = np.empty((len(cdf), 0)).tolist()
    
    idf[dictString] = ''
    censusAreas = []
    global geod 
    geod = Geod(ellps='WGS84')
   
    geoSetup(cdf)
    geoSetup(idf)
   
    for index, row in cdf.iterrows():
        poly_area, poly_perimeter = geod.geometry_area_perimeter(row[cgs])
        poly_area = poly_area*-1
        censusAreas.append(poly_area)
    
    cdf['Area'] = censusAreas
   
    for i in range(len(idf)):
        # Create a new dictionary for tracking isochrones
        iso_dict = defaultdict(list)
        dictionary = {}
        dictionary = check(idf.loc[i ,igs], cdf, cgs, dictionary, key, iso_dict)
        idf.loc[i, dictString] = [dictionary]
        idf.loc[i, 'iso geometries'] = [iso_dict]
        
        for j in range(len(cdf)):
               
            if dictionary.get(cdf.loc[j, key]) != None:
                score = cdf.loc[j, scoreString]
                cdf.loc[j, scoreString] = score + dictionary.get(cdf.loc[j, key])
            
    for i in range(len(cdf)):
        if len(cdf.loc[i, 'polygons in census']) > 0:
            gl = []
            cdf.loc[i, 'number of polygons in census tract'] = len(cdf.loc[i, 'polygons in census'][0])
            toMerge = (cdf.loc[i, 'polygons in census'][0]).copy()
            for tract_key, polygons in toMerge.items():
                gl.append(polygons)
            merged_geom = unary_union(gl)
            merged_area, _ = geod.geometry_area_perimeter(merged_geom)
            merged_area *= -1
            area_val = merged_area / cdf.loc[i,'Area']
            cdf.loc[i, 'percent covered by at least one provider'] = area_val
        
        if len(cdf.loc[i, 'area coverage of each polygon in census']) > 0:
            toAddPerc = 0
            toAdd = (cdf.loc[i, 'area coverage of each polygon in census'][0]).copy()
            for tract_key, percent in toAdd.items():
                toAddPerc = toAddPerc + percent
            cdf.loc[i, 'added percentages'] = toAddPerc


In [ ]:
# function call
appendADS(ILtracts_opt, optom_gdf, 'raw score', 'list of dict', 'geometry', 'isochrones', 'GEOID')

In [ ]:
ILtracts_opt['raw score norm'] = ILtracts_opt['raw score']
ILtracts_opt.loc[ILtracts_opt['raw score norm'] > 1, 'raw score norm'] = 1.0
ILtracts_opt.drop(columns=['polygons in census', 'area coverage of each polygon in census'], inplace=True)
ILtracts_opt['Percentage'] = ILtracts_opt[['raw score norm']]*100

ILtracts_opt2 = ILtracts_opt.loc[(ILtracts_opt['raw score norm'] == 0.0)]
ILtracts_opt2.reset_index(inplace=True)

print(ILtracts_opt.shape)
print(ILtracts_opt2.shape)

In [ ]:
map_1 = KeplerGl(height=600, data={'zero access':ILtracts_opt2, 'ILtractscoverage':ILtracts_opt})
map_1